In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.express as px
from plotly.express.colors import n_colors
import plotly.graph_objects as go
from os.path import join as pjoin

sys.path.append("../")
import circletrack_behavior as ctb
import plotting_functions as pf

In [ ]:
## Settings
project_folder = ['MultiCon_Imaging']
experiment_folders = ['MultiCon_Imaging2', 'MultiCon_Imaging4']
dpath = f'../../{project_folder[0]}'
fig_path = f'../../../Conferences_Workshops/OptoGRC/2024/intermediate_figures'
chance_color = 'darkgrey'
avg_color = 'midnightblue'
subject_color = 'darkgrey'
mouse_colors = ['midnightblue', 'darkred', 'darkorchid', 'darkturquoise']
session_list = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]

### Linear track rewards across days.

In [ ]:
## Linear track behavior
data_of_interest = 'lin_behav' ## one of behav, aligned_minian, lin_behav
lin_dict = {'mouse': [], 'experiment': [], 'day': [], 'rewards': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                lin_dict['mouse'].append(mouse)
                lin_dict['experiment'].append(behav['cohort'].unique()[0])
                lin_dict['day'].append(idx+1)
                lin_dict['rewards'].append(np.sum(behav['water']))
lin_df = pd.DataFrame(lin_dict)

In [ ]:
## Plot rewards across days on linear track
fig = pf.plot_behavior_across_days(lin_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()

### Circle track lick accuracy and rewards across days.

In [ ]:
## Circle track behavior
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
circ_dict = {'mouse': [], 'experiment': [], 'day': [], 'rewards': [], 'percent_correct': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                behav = behav[~behav['probe']] ## exclude probe
                if 'MultiCon_Imaging2' in exp_path:
                    lick_thresh = 3
                else:
                    lick_thresh = 5
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]    
                pc = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=lick_thresh, by_trials=False)
                circ_dict['mouse'].append(mouse)
                circ_dict['experiment'].append(behav['cohort'].unique()[0])
                circ_dict['day'].append(idx+1)
                circ_dict['rewards'].append(np.sum(behav['water']))
                circ_dict['percent_correct'].append(pc)
ct_df = pd.DataFrame(circ_dict)

In [ ]:
## Plot percent correct on circle track
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='percent_correct', groupby_var=['day'], plot_transitions=[5.5, 10.5, 15.5], plot_datapoints=False,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=True, transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))

In [ ]:
## Plot rewards across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=[5.5, 10.5, 15.5], plot_datapoints=False,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False, transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.show()

In [ ]:
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
lick_dict_probe = {'mouse': [], 'experiment': [], 'session': [], 'day': [], 'num_licks': [], 'probe_acc': [], 'session_acc': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                if any(behav['probe']):
                    behav_probe = behav[behav['probe']]
                    behav_no_probe = behav[~behav['probe']]
                    if 'MultiCon_Imaging2' in exp_path:
                        lick_thresh = 3
                    else:
                        lick_thresh = 5
                    reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                    percent_correct = ctb.lick_accuracy(behav_probe, port_one=reward_one, port_two=reward_two, lick_threshold=lick_thresh, by_trials=False)
                    session_pc = ctb.lick_accuracy(behav_no_probe, port_one=reward_one, port_two=reward_two, lick_threshold=lick_thresh, by_trials=False)
                    lick_dict_probe['mouse'].append(mouse)
                    lick_dict_probe['experiment'].append(behav['cohort'].unique()[0])
                    lick_dict_probe['day'].append(idx+1)
                    lick_dict_probe['session'].append(np.unique(behav['session'])[0])
                    lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                    lick_dict_probe['probe_acc'].append(percent_correct)
                    lick_dict_probe['session_acc'].append(session_pc)
                else:
                    pass
probe_df = pd.DataFrame(lick_dict_probe)

first_last = pd.DataFrame()
context_list = ['A', 'B', 'C', 'D']
for mouse in probe_df['mouse'].unique():
    mouse_data = probe_df[probe_df['mouse'] == mouse]
    index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
    index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
    sub_data = mouse_data.loc[index_list, :]
    sub_data.insert(0, 'day_type', 'First')
    sub_data_two = mouse_data.loc[index_list_two, :]
    sub_data_two.insert(0, 'day_type', 'Last')
    first_last = pd.concat([first_last, sub_data, sub_data_two])
avg_combined = first_last.groupby(['day_type', 'session'], as_index=False).agg({'probe_acc': ['mean', 'sem']})

In [ ]:
## Plot probe performance
fig = pf.custom_graph_template(x_title='', y_title='', width=800, rows=1, columns=len(context_list), 
                               titles=['A', 'B', 'C', 'D'], shared_x=True, shared_y=True)
for idx, session in enumerate(context_list):
    plot_data = avg_combined[avg_combined['session'] == session]
    fig.add_trace(go.Scatter(x=plot_data['day_type'], y=plot_data['probe_acc']['mean'],
                                error_y=dict(type='data', array=plot_data['probe_acc']['sem'], thickness=1.5, width=8), 
                                line_color=avg_color, showlegend=False), row=1, col=idx+1)
# for mouse in first_last['mouse'].unique():
#     mdata = first_last[first_last['mouse'] == mouse]
#     for idx, context in enumerate(mdata['session'].unique()):
#         pdata = mdata[mdata['session'] == context]
#         fig.add_trace(go.Scatter(x=pdata['day_type'], y=pdata['probe_acc'], mode='lines', line_color=chance_color,
#                                  line_width=1, opacity=0.7, showlegend=False, name=mouse), row=1, col=idx+1)

fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(title='Probe Accuracy (%)', range=[0, 100], col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc.png'))

In [ ]:
lick_dict = {'mouse': [], 'experiment': [], 'session': [], 'day': [], 'num_licks': [], 'probe_acc': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                if any(behav['probe']):
                    behav_probe = behav[behav['probe']]
                    behav_no_probe = behav[~behav['probe']]
                    if 'MultiCon_Imaging2' in exp_path:
                        lick_thresh = 3
                    else:
                        lick_thresh = 5
                    reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                    percent_correct = ctb.lick_accuracy(behav_probe, port_one=reward_one, port_two=reward_two, lick_threshold=lick_threshold, by_trials=True)
                    lick_dict['mouse'].append(mouse)
                    lick_dict['experiment'].append(behav['cohort'].unique()[0])
                    lick_dict['day'].append(idx+1)
                    lick_dict['session'].append(np.unique(behav['session'])[0])
                    lick_dict['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                    lick_dict['probe_acc'].append(percent_correct)
        else:
            pass
probe_trial_df = pd.DataFrame(lick_dict)

first_last_trial = pd.DataFrame()
context_list = ['A', 'B', 'C', 'D']
for mouse in probe_trial_df['mouse'].unique():
    mouse_data = probe_trial_df[probe_trial_df['mouse'] == mouse]
    index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
    index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
    sub_data = mouse_data.loc[index_list, :]
    sub_data.insert(0, 'day_type', 'First')
    sub_data_two = mouse_data.loc[index_list_two, :]
    sub_data_two.insert(0, 'day_type', 'Last')
    first_last_trial = pd.concat([first_last_trial, sub_data, sub_data_two])

In [ ]:
fig = pf.custom_graph_template(x_title='', y_title='', height=1100, width=1000, rows=4, columns=4,
                               titles=[f'Context {x}, {y}' for x in context_list for y in first_last_trial['mouse'].unique()], 
                               shared_y=True, shared_x=True)
for idx, mouse in enumerate(first_last_trial['mouse'].unique()):
    mdata = first_last_trial[first_last_trial['mouse'] == mouse]
    for sess_idx, session in enumerate(mdata['session'].unique()):
        sdata = mdata[mdata['session'] == session]
    
        for day in sdata['day']:
            if day % 5 == 0:
                marker_color = 'midnightblue'
                linetype = 'solid'
            else:
                marker_color = 'darkgrey'
                linetype = 'dash'

            if sess_idx == 0:
                row, col = 1, idx + 1
            elif sess_idx == 1:
                row, col = 2, idx + 1
            elif sess_idx == 2:
                row, col = 3, idx + 1
            else:
                row, col = 4, idx + 1

            day_data = sdata[sdata['day'] == day]
            x_data = np.arange(1, len(day_data['probe_acc'].values[0])+1)
            fig.add_trace(go.Scatter(x=x_data, y=day_data['probe_acc'].values[0], mode='lines+markers', line=dict(dash=linetype), showlegend=False,
                                    line_color=marker_color, name=day_data['day_type'].values[0], legendgroup=day_data['day_type'].values[0]), row=row, col=col)
fig['data'][0]['showlegend'] = True
fig['data'][1]['showlegend'] = True
fig.update_yaxes(title='Probe Accuracy (%)', col=1)
fig.update_xaxes(title='Trial', row=4)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_performance_across_trials_per_mouse.png'))

### Plot lick accuracy across trials for each session.

In [ ]:
## Plot percent correct across trials for each session
lick_threshold = 5
bin_size = 4
opacity = 0.7
fig = pf.custom_graph_template(x_title='', y_title='', height=1000, width=1000, rows=4, columns=5, 
                               shared_x=True, shared_y=True, titles=session_list)
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for ind, mouse in enumerate(os.listdir(exp_path)):
            mpath = pjoin(exp_path, mouse)
            for idx, session in enumerate(os.listdir(mpath)):
                if 'MultiCon_Imaging2' in exp_path:
                    lick_thresh = 3
                else:
                    lick_thresh = 5
                
                behav = pd.read_feather(pjoin(mpath, f'{session}'))
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                pc = ctb.lick_accuracy(behav, reward_one, reward_two, lick_threshold=lick_threshold, by_trials=True)
                binned_pc = ctb.bin_data(pc, bin_size=bin_size)
                x_data = np.arange(1, len(binned_pc)+1) * bin_size
                if idx < 5:
                    row, col = 1, idx + 1
                elif (idx >= 5) & (idx < 10):
                    row, col = 2, idx - 4
                elif (idx >= 10) & (idx < 15):
                    row, col = 3, idx - 9
                elif idx >= 15:
                    row, col = 4, idx - 14

                fig.add_trace(go.Scatter(x=x_data, y=binned_pc, opacity=opacity,
                                        mode='lines', line_color=mouse_colors[ind], 
                                        showlegend=False, name=mouse), row=row, col=col)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_xaxes(title='Trial', row=4)
fig.update_yaxes(title='Lick Accuracy (%)', col=1, dtick=25)
fig.show()

### Signal detection metrics.

In [ ]:
## Correct rejection rate
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
sig_df = pd.DataFrame()
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                behav = behav[~behav['probe']] ## exclude probe
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]    
                signal = pd.DataFrame(ctb.dprime_metrics(behav, mouse, day=idx+1, reward_ports=[reward_one, reward_two], forward_reverse='forward'))
                signal['experiment'] = behav['cohort'].unique()[0]
                sig_df = pd.concat([sig_df, signal], ignore_index=True)

In [ ]:
## Plot correct rejection rate
corr_rej = sig_df.groupby(['day', 'mouse'], as_index=False).agg({'CR': 'mean'})
fig = pf.plot_behavior_across_days(corr_rej, x_var='day', y_var='CR', groupby_var=['day'], plot_transitions=[5.5, 10.5, 15.5],
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False, transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Correct Rejection Rate', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 1])
fig.show()